### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Connect to git

In [2]:
!git clone https://github.com/Nonempirical/protoInterpretation.git
%cd protoInterpretation
!pip install -e .


Cloning into 'protoInterpretation'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 122 (delta 59), reused 98 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (122/122), 53.04 KiB | 17.68 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/protoInterpretation
Obtaining file:///content/protoInterpretation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for protoInterpretation


### Load dataset

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import drive
from src.protoInterpretation import find_runs_directory, scan_runs

# Mount Drive
drive.mount('/content/drive', force_remount=False)

# Find runs directory
PROJECT_FOLDER_NAME = "protoInterpretation-runs"
search_roots = ["/content/drive/MyDrive", "/content/drive/shared-with-me"]

BASE_RUN_DIR = None
for root in search_roots:
    found = find_runs_directory(root, folder_name=PROJECT_FOLDER_NAME)
    if found:
        BASE_RUN_DIR = found
        break

if BASE_RUN_DIR is None:
    raise RuntimeError(f"Could not find '{PROJECT_FOLDER_NAME}' folder")

# Scan runs
runs = scan_runs(BASE_RUN_DIR)
run_names = sorted(runs.keys())

print(f"Found {len(run_names)} runs")

# --- NEW: "Run all prompts" option ---
RUN_ALL = "__RUN_ALL__"

# Use (label, value) pairs so the displayed text can be friendly
run_options = [("", ""), ("Run all prompts", RUN_ALL)] + [(name, name) for name in run_names]

dropdowns = [
    widgets.Dropdown(
        options=run_options,
        value="",
        description=f"Run {chr(65+i)}:",
        layout=widgets.Layout(width="300px")
    )
    for i in range(10)
]

def _on_any_dropdown_change(change):
    if change.get("name") != "value":
        return
    all_selected = any(dd.value == RUN_ALL for dd in dropdowns)
    if all_selected:
        # Optional UI polish: disable other dropdowns when "Run all prompts" is active
        for dd in dropdowns:
            dd.disabled = (dd.value != RUN_ALL)
    else:
        for dd in dropdowns:
            dd.disabled = False

for dd in dropdowns:
    dd.observe(_on_any_dropdown_change, names="value")

def get_selected_run_names():
    """Call this in later cells to get the final list of runs to process."""
    if any(dd.value == RUN_ALL for dd in dropdowns):
        return run_names  # all runs
    # otherwise: unique, non-empty selections (keep order)
    selected = []
    seen = set()
    for dd in dropdowns:
        v = dd.value
        if v and v not in seen:
            seen.add(v)
            selected.append(v)
    return selected

rows = [widgets.HBox(dropdowns[i:i+2]) for i in range(0, len(dropdowns), 2)]

display(widgets.VBox([
    widgets.HTML("<h3>Select Runs to Visualize</h3>"),
    *rows
]))

### Entropy plots

In [3]:
# Open + Closed entropy plots with COMMON x/y axis limits + APA-style titles
# (and x-axis starts slightly before 0)

import numpy as np
import matplotlib.pyplot as plt

from src.protoInterpretation import run_name_to_display, load_run_by_name

if "runs" not in globals():
    raise RuntimeError("`runs` is not defined. Run the scan_runs cell first.")

PROMPT_PAIRS = [
    ("A bat is", "A pencil is"),
    ("The woman in the blue dress", "Napoleon is"),
    ("I saw", "Photosynthesis is the process where"),
    ("Something happens when there exists", "The declaration of Independence formally"),
    ("The man", "Photosynthesis is"),
    ("The man in the street", "Erosion is"),
]
OPEN_PROMPTS = [p[0] for p in PROMPT_PAIRS]
CLOSED_PROMPTS = [p[1] for p in PROMPT_PAIRS]

def _norm(s: str) -> str:
    return " ".join(str(s).strip().lower().split())

def set_apa_title(fig, title_text, *, y=0.98):
    title = str(title_text).strip()
    if title.endswith("."):
        title = title[:-1]
    fig.text(0.01, y, title, ha="left", va="top", fontsize=11, fontstyle="italic")

open_keys = [_norm(p) for p in OPEN_PROMPTS]
closed_keys = [_norm(p) for p in CLOSED_PROMPTS]

# Candidate runs: if dropdowns exists and has selections -> restrict; else all runs
candidate_run_names = list(runs.keys())
if "dropdowns" in globals():
    picked = [getattr(d, "value", "") for d in dropdowns if getattr(d, "value", "")]
    picked = list(dict.fromkeys([p for p in picked if p]))
    if picked:
        candidate_run_names = picked

# Pick latest run per prompt (by timestamp sort in folder name)
runs_for_key = {k: [] for k in set(open_keys + closed_keys)}
for rn in candidate_run_names:
    k = _norm(run_name_to_display(rn))
    if k in runs_for_key:
        runs_for_key[k].append(rn)

chosen_for_key = {k: max(v) for k, v in runs_for_key.items() if v}

open_runs = [chosen_for_key[k] for k in open_keys if k in chosen_for_key]
closed_runs = [chosen_for_key[k] for k in closed_keys if k in chosen_for_key]
if not open_runs or not closed_runs:
    raise RuntimeError("Missing runs for open or closed prompts (check run names / prompt text matching).")

# Load metrics for both groups
metrics_open = {}
metrics_closed = {}

for k in open_keys:
    rn = chosen_for_key.get(k)
    if rn:
        _, m = load_run_by_name(rn, runs, compute_metrics=True)
        metrics_open[rn] = m

for k in closed_keys:
    rn = chosen_for_key.get(k)
    if rn:
        _, m = load_run_by_name(rn, runs, compute_metrics=True)
        metrics_closed[rn] = m

# Shared axis limits across BOTH plots
all_means = []
max_T = 0
for m in list(metrics_open.values()) + list(metrics_closed.values()):
    y = m.entropy.mean
    if len(y):
        all_means.append(y)
        max_T = max(max_T, len(y))

y_all = np.concatenate(all_means)
y_min, y_max = float(y_all.min()), float(y_all.max())
y_pad = 0.05 * (y_max - y_min) if y_max > y_min else 0.25
Y_LIM = (y_min - y_pad, y_max + y_pad)

# x starts before 0 for readability
X_LIM = (-0.5, max_T - 1)

def plot_group(metrics_dict, title_text):
    fig, ax = plt.subplots(figsize=(7, 4))

    for rn, m in metrics_dict.items():
        label = run_name_to_display(rn)
        y = m.entropy.mean
        ax.plot(np.arange(len(y)), y, linewidth=2, alpha=0.9, label=label)

    ax.set_xlim(*X_LIM)
    ax.set_ylim(*Y_LIM)
    ax.set_xlabel("Step")
    ax.set_ylabel("Entropy (bits)")
    ax.set_title("")  # title handled by set_apa_title
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best", fontsize=8, frameon=True)

    fig.tight_layout(rect=[0, 0, 1, 0.92])
    set_apa_title(fig, title_text)
    plt.show()

plot_group(metrics_open, "Predictive Entropy Across Generation Steps for Open Prompts")
plot_group(metrics_closed, "Predictive Entropy Across Generation Steps for Closed Prompts")

RuntimeError: `runs` is not defined. Run the scan_runs cell first.

### Create UMAP's

In [ ]:
import os
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Prefer installed package name; fall back to repo import
try:
    from src.protoInterpretation import (
        run_name_to_display,
        load_embeddings_from_runs,
        compute_global_umap,
    )
except ImportError:
    from src.protoInterpretation import (
        run_name_to_display,
        load_embeddings_from_runs,
        compute_global_umap,
    )

# ------------------------------------------------------------
# MUST exist from previous cells:
# - runs (dict from scan_runs)
# ------------------------------------------------------------
if "runs" not in globals():
    raise RuntimeError("`runs` is not defined. Run the scan_runs cell first.")

# ------------------------------------------------------------
# Table X (Open first, Closed second)
# ------------------------------------------------------------
PROMPT_PAIRS = [
    ("A bat is", "A pencil is"),
    ("The woman in the blue dress", "Napoleon is"),
    ("I saw", "Photosynthesis is the process where"),
    ("Something happens when there exists", "The declaration of Independence formally"),
    ("The man", "Photosynthesis is"),
    ("The man in the street", "Erosion is"),
]
OPEN_PROMPTS = [p[0] for p in PROMPT_PAIRS]
CLOSED_PROMPTS = [p[1] for p in PROMPT_PAIRS]

def _norm(s: str) -> str:
    return " ".join(str(s).strip().lower().split())

open_keys = [_norm(p) for p in OPEN_PROMPTS]
closed_keys = [_norm(p) for p in CLOSED_PROMPTS]
all_keys_in_pair_order = [k for pair in PROMPT_PAIRS for k in (_norm(pair[0]), _norm(pair[1]))]

key_to_prompt = {k: p for k, p in zip(open_keys, OPEN_PROMPTS)}
key_to_prompt.update({k: p for k, p in zip(closed_keys, CLOSED_PROMPTS)})

key_to_group = {k: "open" for k in open_keys}
key_to_group.update({k: "closed" for k in closed_keys})

# ------------------------------------------------------------
# Styling (consistent)
# ------------------------------------------------------------
OPEN_COLOR = "#1f77b4"    # blue
CLOSED_COLOR = "#ff7f0e"  # orange
POINT_ALPHA = 0.65
POINT_SIZE = 8

def set_apa_title(fig, title_text, *, y=0.98):
    title = str(title_text).strip()
    if title.endswith("."):
        title = title[:-1]
    fig.text(0.01, y, title, ha="left", va="top", fontsize=11, fontstyle="italic")

# ------------------------------------------------------------
# UMAP settings
# ------------------------------------------------------------
UMAP_STEP_WANTED = 31
UMAP_METRIC = "cosine"
UMAP_SEED = 42
L2_NORMALIZE = True
PCA_COMPONENTS = 50
UMAP_N_NEIGHBORS = 15
UMAP_MIN_DIST = 0.1

# ------------------------------------------------------------
# Candidate runs: if dropdowns exists and has selections -> restrict; else all runs
# ------------------------------------------------------------
candidate_run_names = list(runs.keys())
if "dropdowns" in globals():
    picked = [getattr(d, "value", "") for d in dropdowns if getattr(d, "value", "")]
    picked = list(dict.fromkeys([p for p in picked if p]))
    if picked:
        candidate_run_names = picked
        print(f"Using {len(candidate_run_names)} candidate runs (from dropdowns).")
    else:
        print(f"Using {len(candidate_run_names)} candidate runs (all runs).")
else:
    print(f"Using {len(candidate_run_names)} candidate runs (all runs).")

# ------------------------------------------------------------
# Match latest run per prompt (by timestamp sorting in run folder name)
# ------------------------------------------------------------
target_keys = set(open_keys + closed_keys)
runs_for_key = {k: [] for k in target_keys}

for rn in candidate_run_names:
    disp = run_name_to_display(rn)
    k = _norm(disp)
    if k in runs_for_key:
        runs_for_key[k].append(rn)

chosen_for_key = {}
for k, lst in runs_for_key.items():
    if lst:
        chosen_for_key[k] = max(lst)  # newest run

open_runs = [chosen_for_key[k] for k in open_keys if k in chosen_for_key]
closed_runs = [chosen_for_key[k] for k in closed_keys if k in chosen_for_key]
all_runs_pair_order = [chosen_for_key[k] for k in all_keys_in_pair_order if k in chosen_for_key]

if not all_runs_pair_order:
    raise RuntimeError("No matching runs found for Table X prompts. Check run names / prompts.")

print(f"Matched runs: open={len(open_runs)}, closed={len(closed_runs)}, total={len(all_runs_pair_order)}")

run_to_group = {}
run_to_prompt = {}
for k in open_keys:
    rn = chosen_for_key.get(k)
    if rn:
        run_to_group[rn] = "open"
        run_to_prompt[rn] = key_to_prompt[k]
for k in closed_keys:
    rn = chosen_for_key.get(k)
    if rn:
        run_to_group[rn] = "closed"
        run_to_prompt[rn] = key_to_prompt[k]

# ------------------------------------------------------------
# Output folder
# ------------------------------------------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = f"umap_three_views_{timestamp}"
os.makedirs(output_folder, exist_ok=True)
print(f"📁 Saving plots to: {output_folder}/")

# ------------------------------------------------------------
# Load embeddings and compute GLOBAL UMAP @ step 31 (shared space for all 3 figures)
# ------------------------------------------------------------
emb_per_run, min_T, D = load_embeddings_from_runs(all_runs_pair_order, runs, load_text_per_step=False)

step_idx = UMAP_STEP_WANTED
if step_idx >= min_T:
    print(f"⚠️ Requested step {UMAP_STEP_WANTED}, but min_T={min_T}. Using step {min_T-1} instead.")
    step_idx = min_T - 1

# Slice to single step: [N, 1, D]
sliced = {rn: emb[:, step_idx:step_idx+1, :] for rn, emb in emb_per_run.items()}
pca_components = min(PCA_COMPONENTS, D)

E_2d, run_labels, step_labels, chain_indices = compute_global_umap(
    sliced,
    max_steps=1,
    pca_components=pca_components,
    umap_n_neighbors=UMAP_N_NEIGHBBORS if 'UMAP_N_NEIGHBBORS' in globals() else UMAP_N_NEIGHBORS,
    umap_min_dist=UMAP_MIN_DIST,
    umap_random_state=UMAP_SEED,
    umap_metric=UMAP_METRIC,
    l2_normalize=L2_NORMALIZE,
)

# --- NEW: shared axis limits (same x/y across all three figures) ---
x_min, x_max = float(E_2d[:, 0].min()), float(E_2d[:, 0].max())
y_min, y_max = float(E_2d[:, 1].min()), float(E_2d[:, 1].max())
x_pad = 0.05 * (x_max - x_min) if x_max > x_min else 1.0
y_pad = 0.05 * (y_max - y_min) if y_max > y_min else 1.0
X_LIM = (x_min - x_pad, x_max + x_pad)
Y_LIM = (y_min - y_pad, y_max + y_pad)

run_labels_arr = np.asarray(run_labels)
types = np.array([run_to_group.get(rn, "unknown") for rn in run_labels_arr])

mask_open = types == "open"
mask_closed = types == "closed"
mask_unknown = types == "unknown"

# ------------------------------------------------------------
# Figure 1: All prompts, color-coded Open vs Closed
# ------------------------------------------------------------
fig1, ax1 = plt.subplots(figsize=(8, 6))
ax1.scatter(E_2d[mask_open, 0], E_2d[mask_open, 1], s=POINT_SIZE, alpha=POINT_ALPHA, color=OPEN_COLOR, label="Open")
ax1.scatter(E_2d[mask_closed, 0], E_2d[mask_closed, 1], s=POINT_SIZE, alpha=POINT_ALPHA, color=CLOSED_COLOR, label="Closed")
if mask_unknown.any():
    ax1.scatter(E_2d[mask_unknown, 0], E_2d[mask_unknown, 1], s=POINT_SIZE, alpha=0.35, color="gray", label="Unknown")

ax1.set_xlim(*X_LIM)
ax1.set_ylim(*Y_LIM)
ax1.set_aspect("equal", adjustable="box")  # optional but helps comparison

ax1.set_xlabel("UMAP 1")
ax1.set_ylabel("UMAP 2")
ax1.set_title("")
ax1.grid(True, alpha=0.25)
ax1.legend(loc="best", frameon=True)
fig1.tight_layout(rect=[0, 0, 1, 0.92])
set_apa_title(fig1, f"Semantic Space at Step {step_idx} by Prompt Type")

fn1 = f"umap_step_{step_idx:02d}_by_type.png"
fig1.savefig(os.path.join(output_folder, fn1), dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig1)
print(f"  ✓ Saved: {fn1}")

# ------------------------------------------------------------
# Figure 2: OPEN prompts only (one color per prompt)
# ------------------------------------------------------------
fig2, ax2 = plt.subplots(figsize=(8, 6))
open_run_set = set(open_runs)
run_labels_unique = list(dict.fromkeys(run_labels))

palette = plt.cm.tab10.colors
open_names = [rn for rn in run_labels_unique if rn in open_run_set]
open_color_map = {rn: palette[i % len(palette)] for i, rn in enumerate(open_names)}

for rn in open_names:
    m = run_labels_arr == rn
    ax2.scatter(E_2d[m, 0], E_2d[m, 1], s=POINT_SIZE, alpha=POINT_ALPHA, color=open_color_map[rn], label=run_to_prompt.get(rn, rn))

ax2.set_xlim(*X_LIM)
ax2.set_ylim(*Y_LIM)
ax2.set_aspect("equal", adjustable="box")

ax2.set_xlabel("UMAP 1")
ax2.set_ylabel("UMAP 2")
ax2.set_title("")
ax2.grid(True, alpha=0.25)
ax2.legend(loc="best", fontsize=8, frameon=True)
fig2.tight_layout(rect=[0, 0, 1, 0.92])
set_apa_title(fig2, f"Semantic Space at Step {step_idx} for Open Prompts")

fn2 = f"umap_step_{step_idx:02d}_open_prompts.png"
fig2.savefig(os.path.join(output_folder, fn2), dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig2)
print(f"  ✓ Saved: {fn2}")

# ------------------------------------------------------------
# Figure 3: CLOSED prompts only (one color per prompt)
# ------------------------------------------------------------
fig3, ax3 = plt.subplots(figsize=(8, 6))
closed_run_set = set(closed_runs)
closed_names = [rn for rn in run_labels_unique if rn in closed_run_set]
closed_color_map = {rn: palette[i % len(palette)] for i, rn in enumerate(closed_names)}

for rn in closed_names:
    m = run_labels_arr == rn
    ax3.scatter(E_2d[m, 0], E_2d[m, 1], s=POINT_SIZE, alpha=POINT_ALPHA, color=closed_color_map[rn], label=run_to_prompt.get(rn, rn))

ax3.set_xlim(*X_LIM)
ax3.set_ylim(*Y_LIM)
ax3.set_aspect("equal", adjustable="box")

ax3.set_xlabel("UMAP 1")
ax3.set_ylabel("UMAP 2")
ax3.set_title("")
ax3.grid(True, alpha=0.25)
ax3.legend(loc="best", fontsize=8, frameon=True)
fig3.tight_layout(rect=[0, 0, 1, 0.92])
set_apa_title(fig3, f"Semantic Space at Step {step_idx} for Closed Prompts")

fn3 = f"umap_step_{step_idx:02d}_closed_prompts.png"
fig3.savefig(os.path.join(output_folder, fn3), dpi=150, bbox_inches="tight", facecolor="white")
plt.close(fig3)
print(f"  ✓ Saved: {fn3}")

# ------------------------------------------------------------
# Download
# ------------------------------------------------------------
print("\n⬇️ Downloading 3 files...")
for fn in [fn1, fn2, fn3]:
    files.download(os.path.join(output_folder, fn))

print("\n✅ Done!")

### Create horizon widht graphs

In [ ]:
# Plot pairwise cosine distance (horizon width) for selected Open vs Closed prompts
# FIRST 5 steps (0..4): one figure for MEAN, one for MAX

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from src.protoInterpretation import run_name_to_display, load_run_by_name

if "runs" not in globals():
    raise RuntimeError("`runs` is not defined. Run the scan_runs cell first.")

PROMPT_PAIRS = [
    ("A bat is", "A pencil is"),
    ("The woman in the blue dress", "Napoleon is"),
    ("I saw", "Photosynthesis is the process where"),
    ("Something happens when there exists", "The declaration of Independence formally"),
    ("The man", "Photosynthesis is"),
    ("The man in the street", "Erosion is"),
]

OPEN_COLOR = "#1f77b4"
CLOSED_COLOR = "#ff7f0e"
OPEN_STYLE = dict(color=OPEN_COLOR, linestyle="-", linewidth=2, alpha=0.9)
CLOSED_STYLE = dict(color=CLOSED_COLOR, linestyle="--", linewidth=2, alpha=0.9)

def _norm(s: str) -> str:
    return " ".join(str(s).strip().lower().split())

open_keys = [_norm(p[0]) for p in PROMPT_PAIRS]
closed_keys = [_norm(p[1]) for p in PROMPT_PAIRS]
key_to_type = {k: "open" for k in open_keys}
key_to_type.update({k: "closed" for k in closed_keys})

# Restrict candidate runs to dropdown selection if present; else all runs
candidate_run_names = list(runs.keys())
if "dropdowns" in globals():
    picked = [getattr(d, "value", "") for d in dropdowns if getattr(d, "value", "")]
    picked = list(dict.fromkeys([p for p in picked if p]))
    if picked:
        candidate_run_names = picked

# Pick latest run per prompt (by timestamp sort in folder name)
runs_for_key = {k: [] for k in set(open_keys + closed_keys)}
for rn in candidate_run_names:
    k = _norm(run_name_to_display(rn))
    if k in runs_for_key:
        runs_for_key[k].append(rn)

chosen_for_key = {k: max(v) for k, v in runs_for_key.items() if v}

missing = [k for k in set(open_keys + closed_keys) if k not in chosen_for_key]
if missing:
    print("⚠️ Missing runs for:", missing)

# Load metrics
metrics_for_key = {}
for k, rn in chosen_for_key.items():
    _, m = load_run_by_name(rn, runs, compute_metrics=True)
    metrics_for_key[k] = m

T_SHOW = 5  # steps 0..4

def plot_width(which: str):
    fig, ax = plt.subplots(figsize=(8, 4))
    for k, m in metrics_for_key.items():
        y_full = m.width.mean_dist if which == "mean" else m.width.max_dist
        y = y_full[:T_SHOW] if len(y_full) >= T_SHOW else y_full
        g = key_to_type.get(k, "unknown")
        if g == "open":
            ax.plot(np.arange(len(y)), y, **OPEN_STYLE)
        elif g == "closed":
            ax.plot(np.arange(len(y)), y, **CLOSED_STYLE)

    ax.set_xlabel("Step (first 5)")
    ax.set_ylabel("Cosine distance")
    ax.set_title(f"Pairwise Cosine Distance ({which.upper()}) for Open vs Closed Prompts (Steps 0–4)")
    ax.grid(True, alpha=0.3)

    handles = [
        Line2D([0], [0], **OPEN_STYLE, label="Open prompts"),
        Line2D([0], [0], **CLOSED_STYLE, label="Closed prompts"),
    ]
    ax.legend(handles=handles, loc="best", frameon=True)

    plt.tight_layout()
    plt.show()

plot_width("mean")
plot_width("max")

# Print all sqeunces from a prompt


In [ ]:
# Prefer installed package name; fall back to repo import
try:
    from protoInterpretation import load_run_by_name
except ImportError:
    from src.protoInterpretation import load_run_by_name

import ipywidgets as widgets
from IPython.display import display

# Pick selected run from dropdowns
run_name = next((d.value for d in dropdowns if d.value), None)
if not run_name:
    raise RuntimeError("Select at least 1 run in the dropdowns above.")

batch, _ = load_run_by_name(run_name, runs, compute_metrics=False)

# Decide how many chains exist
N = None
if batch.text_sequences is not None and len(batch.text_sequences) > 0:
    N = len(batch.text_sequences)
elif batch.text_per_step is not None and len(batch.text_per_step) > 0:
    N = len(batch.text_per_step)
else:
    raise RuntimeError(
        "This run has no decoded text (text_sequences/text_per_step missing). "
        "Re-run sampling with a newer version or decode token_ids with a tokenizer."
    )

print(f"Run: {run_name} | N={N} chains")

# How many chains to render at once
K = 20  # change this

start_slider = widgets.IntSlider(min=0, max=max(0, N - 1), step=1, description="Start")
out = widgets.Output()

def get_full_chain(i: int) -> str:
    # Preferred: full final decoded chain
    if batch.text_sequences is not None and i < len(batch.text_sequences):
        return batch.text_sequences[i]

    # Fallback: last step in per-step cumulative text
    if batch.text_per_step is not None and i < len(batch.text_per_step) and len(batch.text_per_step[i]) > 0:
        return batch.text_per_step[i][-1]

    return "[No decoded full-chain text available]"

def render(_=None):
    out.clear_output()
    with out:
        start = start_slider.value
        end = min(start + K, N)
        for i in range(start, end):
            print(f"{get_full_chain(i)}")
            print()

start_slider.observe(render, names="value")
render()
display(start_slider, out)